# Forte Tutorial 1.04: Implementing coupled cluster theory with Forte's sparse operator class

---

In this tutorial we will implement coupled cluster theory using Forte's `SparseOperator` and `StateVector` classes.
We will avoid many of the complications due to symmetry and work with molecules in $C_1$ symmetry.

## Outline of coupled cluster theory
Coupled cluster theory assumes the following exponential ansatz for the wave function
$$
| \Psi_\mathrm{CC} \rangle = \exp(\hat{T}) | \Phi \rangle
$$
where $| \Phi \rangle$ is a Slater determinant and $\hat{T}$ is an excitation operator. This operator is a sum of $k$-body excitation operators, where $k$ goes from 1 to a truncation level $n$
$$
\hat{T} = \sum_{k=1}^{n} \hat{T}_k
$$
Each component of $\hat{T}$ is defined as
$$
\hat{T}_k = \frac{1}{(k!)^2} \sum_{ij\cdots}^\mathrm{occ} \sum_{ab\cdots}^\mathrm{vir} t_{ij\cdots}^{ab\cdots} \hat{a}^\dagger_a \hat{a}^\dagger_b \cdots \hat{a}_j \hat{a}_i
$$
or equivalently using only unique terms as
$$
\hat{T}_k = \sum_{i < j < \cdots}^\mathrm{occ} \sum_{a < b < \cdots}^\mathrm{vir} t_{ij\cdots}^{ab\cdots} \hat{a}^\dagger_a \hat{a}^\dagger_b \cdots \hat{a}_j \hat{a}_i
$$

The coupled cluster equations are
$$
E = \langle \Phi | \exp(-\hat{T}) \hat{H} \exp(\hat{T}) | \Phi \rangle
$$
and
$$
0 = \langle \Phi_{ij\cdots}^{ab\cdots} | \exp(-\hat{T}) \hat{H} \exp(\hat{T}) | \Phi \rangle
$$
where $\Phi_{ij\cdots}^{ab\cdots}$ can be any of the determinants obtained by applying $\hat{T}$ to the reference $\Phi$.

## Implementation strategy

Our implementation will be based on a determinantal code. We will represent the coupled cluster wave function and any other state as a linear combination of determinants.
For example, the coupled cluster wave function will be a sum of the form
$$
| \Psi_\mathrm{CC} \rangle = \sum_I C_I |\Phi_I\rangle
$$
where $|\Phi_I\rangle$ is a Slater determinant and $C_I$ its coefficient.
Using Forte's implementation of sparse operators and state vectors we can build all the quantities that will be required to solve the CC equations.

To solve the CC equations we will use a simple update procedure commonly used to implement CC theory. Starting from the condition
$$
0 = \langle \Phi_{ij\cdots}^{ab\cdots} | \exp(-\hat{T}) \hat{H} \exp(\hat{T}) | \Phi \rangle
$$
we can add the term $t_{ij\cdots}^{ab\cdots} \Delta_{ij\cdots}^{ab\cdots}$ to both sides to the above condition and divide by $\Delta_{ij\cdots}^{ab\cdots}$ to obtain,
$$
t_{ij\cdots}^{ab\cdots}  = t_{ij\cdots}^{ab\cdots} + \frac{\langle \Phi_{ij\cdots}^{ab\cdots} | \exp(-\hat{T}) \hat{H} \exp(\hat{T}) | \Phi \rangle}{\Delta_{ij\cdots}^{ab\cdots}}
$$
We can now interpret this equation as an update equation, where we start from a set of amplitudes, plug them in the right hand side of this equation, and obtain the left hand side, which we treat as a new set of amplitudes
$$
t_{ij\cdots}^{ab\cdots}\text{(new)}  = t_{ij\cdots}^{ab\cdots}\text{(old)} + \frac{\langle \Phi_{ij\cdots}^{ab\cdots} | \exp(-\hat{T}) \hat{H} \exp(\hat{T}) | \Phi \rangle}{\Delta_{ij\cdots}^{ab\cdots}}
$$
To implement this equation we will need to evaluate the matrix elements $R_{ij\cdots}^{ab\cdots} = \langle \Phi_{ij\cdots}^{ab\cdots} | \exp(-\hat{T}) \hat{H} \exp(\hat{T}) | \Phi \rangle$.

## Step 1. Import modules and get Hartree–Fock MOs from psi4

We begin by importing psi4, forte, and some useful modules. We also call the `startup()` function in forte

In [ ]:
import itertools
import functools
import time

import psi4
import forte
import forte.utils

forte.startup()

We will use a simple toy model: the H$_4$ molecule. First we will run psi4 using the function `forte.utils.psi4_scf` and compute both the HF and CCSD energy. Note that we run the computation in C$_1$ symmetry by adding `symmetry C1` to the geometry section

In [ ]:
# setup xyz geometry for linear H4
geometry = """
H 0.0 0.0 0.0
H 0.0 0.0 1.0
H 0.0 0.0 2.0
H 0.0 0.0 3.0
symmetry c1
"""

(escf, psi4_wfn) = forte.utils.psi4_scf(geometry,basis='sto-3g',reference='rhf',options={'E_CONVERGENCE' : 1.e-12})

if psi4_wfn.nirrep() != 1:
    raise RuntimeError('This code can only run in C1 symmetry. Add "symmetry C1" in your geometry section.')

eccsd = psi4.energy('ccsd/sto-3g')

print(f'SCF Energy:  {escf:16.12f}')
print(f'CCSD Energy: {eccsd:16.12f}')

## Preparing the Forte objects (integrals, MO space information)
Here we just build all the object we will need in our code. **For convenience we will treat all orbitals as active** and grab integrals from the `ActiveSpaceIntegrals` object

In [ ]:
# pass mo_spaces={} to treat all orbitals as active
forte_objs = forte.utils.prepare_forte_objects(psi4_wfn,mo_spaces={})
ints, as_ints, scf_info, mo_space_info, state_weights_map = forte_objs

## Forming the Hartree–Fock state

Next, we read the number of MOs and alpha/beta electrons from the psi4 wave function object. This data is generated by psi4 when running the Hartree–Fock code. We use this information to build the Hartree–Fock Slater determinant. By this we mean that we define the occupation of the HF determinant. We don't optimize the orbitals here because that is already done in the SCF computation done by psi4

In [ ]:
# get the number of MOs and alpha/beta electrons per irrep
nmo = mo_space_info.size('CORRELATED')
nael = scf_info.doccpi().sum() + scf_info.soccpi().sum()
nbel = scf_info.doccpi().sum()

if nael != nbel:
    raise RuntimeError('The number of alpha and beta electrons must be the same')

# Specify the occupation of the the Hartree–Fock determinant
hfref = forte.Determinant()
# set the occupation numbers of the determinant
for i in range(nael): hfref.set_alfa_bit(i, True)
for i in range(nbel): hfref.set_beta_bit(i, True)        

print(f'Number of orbitals:        {nmo}')
print(f'Number of alpha electrons: {nael}')
print(f'Number of beta electrons:  {nbel}')
print(f'Reference determinant:     {hfref.str(nmo)}')

## Creating the coupled cluster excitation operator

In this example we will generate the CCSD cluster operator
$$
\hat{T} = \sum_{k=1}^{n} \hat{T}_k
$$
Where each component of $\hat{T}$ is defined as
$$
\hat{T}_k = \sum_{i < j < \cdots}^\mathrm{occ} \sum_{a < b < \cdots}^\mathrm{vir} t_{ij\cdots}^{ab\cdots} \hat{a}^\dagger_a \hat{a}^\dagger_b \cdots \hat{a}_j \hat{a}_i
$$
Here we use the `itertools.combinations()` function to generate unique combinations of orbital indices $i < j < \cdots$ and $a < b < \cdots$.
This code breaks down the construction of the excitation operator into several loops.
What this code does is generate all the operators $\hat{a}^\dagger_a \hat{a}^\dagger_b \cdots \hat{a}_j \hat{a}_i$ and pushes them to a `SparseOperator` object.
We initialize the amplitudes to 0.

As we form the operators, we also compute their corresponding denominators
$$
\Delta_{ij\cdots}^{ab\cdots} = \epsilon_a + \epsilon_b + \ldots - \epsilon_i - \epsilon_j - \ldots
$$

It is important to understand how the operators, amplitudes, and denominators are numbered. Since the `SparseOperator` object hold the operators and the corresponding coeffients in a vector according to the order in which they are added, we can associate an integer index ($\mu$) to each operator, that is we can write
$$
\hat{a}^\dagger_a \hat{a}^\dagger_b \cdots \hat{a}_j \hat{a}_i = \hat{\tau}_\mu
$$
where $\hat{\tau}_\mu$ is the operator labeled by index $\mu$.
It is then convinient to store the denominators using the same indexing and to compute the residuals using the same indexing.

In [ ]:
# Prepare the cluster operator (closed-shell case)
occ = list(range(nael))
vir = list(range(nael,nmo))

print(f'Occupied orbitals: {occ}')
print(f'Virtual orbitals:  {vir}')

max_exc = 2

op = forte.SparseOperator()

ea = scf_info.epsilon_a()
eb = scf_info.epsilon_a()

denominators = []

# loop over total excitation level
for n in range(1,max_exc + 1):
    # loop over beta excitation level
    for nb in range(n + 1):
        na = n - nb
        # loop over alpha occupied
        for ao in itertools.combinations(occ, na):
            # loop over alpha virtual
            for av in itertools.combinations(vir, na):
                # loop over beta occupied
                for bo in itertools.combinations(occ, nb):
                    # loop over beta virtual
                    for bv in itertools.combinations(vir, nb):
                        # compute the denominators
                        e_aocc = functools.reduce(lambda x, y: x + ea.get(y),ao,0.0)
                        e_avir = functools.reduce(lambda x, y: x + ea.get(y),av,0.0)                
                        e_bocc = functools.reduce(lambda x, y: x + eb.get(y),bo,0.0)
                        e_bvir = functools.reduce(lambda x, y: x + eb.get(y),bv,0.0)                                       
                        denominators.append(e_aocc + e_bocc - e_bvir - e_avir)
                        
                        # create an operator from a list of tuples (creation, alpha, orb) where
                        #   creation : bool (true = creation, false = annihilation)
                        #   alpha    : bool (true = alpha, false = beta)
                        #   orb      : int  (the index of the mo)
                        l = [] # a list to hold the operator triplets
                        for i in ao: l.append((False,True,i)) # alpha occupied
                        for i in bo: l.append((False,False,i)) # beta occupied        
                        for a in reversed(bv): l.append((True,False,a)) # beta virtual                                                                    
                        for a in reversed(av): l.append((True,True,a)) # alpha virtual
                        op.add_term(l,0.0)

print(f'==> Operator <==')
print(f'Number of amplitudes: {op.size()}')
print("\n".join(op.str()))

## The coupled cluster equations
Here we define a function that computes the quantity
$$
\langle \Phi_{ij\cdots}^{ab\cdots} |\exp(-\hat{T}) \hat{H} \exp(\hat{T}) | \Phi \rangle
$$
We compute this quantity in the following steps:
1. Apply $\exp(\hat{T})$ to $| \Phi \rangle$ using a `SparseExp` object (`exp_op`).
This object evaluates $\exp(\hat{T})| \Phi \rangle$ by expanding the exponential using a Taylor series that is truncated to be numerically exact
$$
\exp(\hat{T}) | \Phi \rangle = | \Phi \rangle + \hat{T} | \Phi \rangle + \frac{1}{2}\hat{T}^2 | \Phi \rangle
+ \frac{1}{6}\hat{T}^3| \Phi \rangle + \ldots
$$
2. Apply $\hat{H}$ to $\exp(\hat{T}) | \Phi \rangle$ using a `SparseHamiltonian` object (`ham_op`). This step gives us the quantity
$$
\hat{H} \exp(\hat{T}) | \Phi \rangle
$$
3. Apply $\exp(-\hat{T})$  to $\hat{H}\exp(\hat{T}) | \Phi \rangle$ using the `SparseExp` object. This step gives us the quantity
$$
\exp(-\hat{T}) \hat{H} \exp(\hat{T}) | \Phi \rangle
$$
4. Project $\exp(-\hat{T}) \hat{H} \exp(\hat{T}) | \Phi \rangle$ onto the set of excited determinants
$$
\langle \Phi_{ij\cdots}^{ab\cdots} |\exp(-\hat{T}) \hat{H} \exp(\hat{T}) | \Phi \rangle
$$
Here we use the function `forte.get_projection()` which computes the quantity
$$
R_\mu = \langle \Phi | (\hat{\tau}_\mu)^\dagger \exp(-\hat{T}) \hat{H} \exp(\hat{T}) | \Phi \rangle
$$
and takes careful of $\pm$ sign factors that may arise from the excitation operator $\hat{\tau}_\mu = \hat{a}^\dagger_a \hat{a}^\dagger_b \cdots \hat{a}_j \hat{a}_i$.

In [ ]:
def cc_residual_equations(op,ref,ham_op,exp_op):
    """This function implements the CC residual equation

    Parameters
    ----------
    op : SparseOperator
        The cluster operator
    ref : StateVector
        The reference wave function
    ham_op : SparseHamiltonian
        The Hamiltonian operator
    exp_op : SparseExp
        The exponential operator        

    Returns
    -------
    tuple(list(float),float)
        A tuple with the residual and the energy
    """    
    
    # Step 1. Compute exp(T)|Phi>
    wfn = exp_op.compute(op,ref,scaling_factor=1.0)
    
    # Step 2. Compute H exp(T)|Phi>
    Hwfn = ham_op.compute(wfn,0.0)
    
    # Step 3. Compute exp(-T) H exp(T)|Phi>
    R = exp_op.compute(op,Hwfn,scaling_factor=-1.0)
    
    # Step 4. Project residual onto excited determinants
    residual = forte.get_projection(op,ref,R)
    
    energy = 0.0
    for d,c in ref.items():
        energy += c * R[d];
    return (residual, energy)           

## The update equation
Here we define a function that implements the update equation
$$
t_{\mu}\text{(new)}  = t_{\mu}\text{(old)} + \frac{R_{\mu}}{\Delta_{\mu}}
$$

In [ ]:
def update_amps(op,residual,denominators):
    """This function updates the CC amplitudes

    Parameters
    ----------
    op : SparseOperator
        The cluster operator. The amplitudes will be updates after running this function
    residual : list(float)
        The residual
    denominators : list(float)
        The Møller–Plesset denominators
    """        
    t = op.coefficients()
    # update the amplitudes
    for i in range(op.size()):
        t[i] += residual[i] / denominators[i]
    # push new amplitudes to the T operator
    op.set_coefficients(t)

## The CC procedure

We now have all the ingredients to setup the CC procedure. This code has the following structure:
We compute this quantity in the following steps:
1. Initialize $\hat{T}$ and the operators that implement the Hamiltonian and the exponential
1. Evaluate the CC residual equations
1. Update the amplitudes
1. Check for convergence of the energy. If not converged go to Step 2., otherwise, stop and print the final energy

In [ ]:
import numpy as np

start = time.time()

# initialize T = 0
t = [0.0] * op.size()
op.set_coefficients(t)

# initalize E = 0
old_e = 0.0

# create the reference, the Hamiltonian, and the exponential operator
ref = forte.StateVector({hfref :1.0})
ham_op = forte.SparseHamiltonian(as_ints)
exp_op = forte.SparseExp()

print('=================================================================')
print('   Iteration     Energy (Eh)       Delta Energy (Eh)    Time (s)')   
print('-----------------------------------------------------------------')

# define the convergence criterion
e_convergence = 1.0e-12

# set the maximum number of CC iterations
max_cc_iter = 100

for iter in range(max_cc_iter):
    # 1. evaluate the CC residual equations
    residual, e = cc_residual_equations(op,ref,ham_op,exp_op)
    
    # 2. update the CC equations
    update_amps(op,residual,denominators)
        
    # 3. print information
    print(f'{iter:9d} {e:20.12f} {e - old_e:20.12f} {time.time() - start:11.3f}')          
        
    # 4. check for convergence of the energy
    if abs(e - old_e) < e_convergence:
        break
    old_e = e
    
print('=================================================================')

print(f' CCSD energy (forte): {e:20.12f} [Eh]')
print(f' CCSD energy (psi4):  {eccsd:20.12f} [Eh]')

In [ ]:
forte.cleanup() # necessary to close ambit

If your final energy is equal to about -2.166379520 then congratulations! You have just implemented CCSD!

## Extensions

The code above can be extended in several ways:
1. Implement higher-order CC theory. For example, CCSDT and CCSDTQ.
1. Implement unitary CCSD, replacing the exponential $\exp(\hat{T})$ with the unitary operator $\exp(\hat{T} - \hat{T}^\dagger)$.
1. Implement DIIS to speed up convergence of the CC procedure
1. Take into account symmetry and only include those components of $\hat{T}$ that are total symmetric.
1. Implement EOM-CC theory.